# **Transfer Learning in Deep Learning**

`What is transfer learning and how it works?`

The reuse of a pre-trained model on a new problem is known as transfer learning in machine learning. A machine uses the knowledge learned from a prior assignment to increase prediction about a new task in transfer learning. You could, for example, use the information gained during training to distinguish beverages when training a classifier to predict whether an image contains cuisine.

`How it works?`

In computer vision, neural networks typically aim to detect edges in the first layer, forms in the middle layer, and task-specific features in the latter layers.

The early and central layers are employed in transfer learning, and the latter layers are only retrained. It makes use of the labelled data from the task it was trained on.

`Why Should You Use Transfer Learning?`

Transfer learning offers a number of advantages, the most important of which are reduced training time, improved neural network performance (in most circumstances), and the absence of a large amount of data.

To train a neural model from scratch, a lot of data is typically needed, but access to that data isn’t always possible – this is when transfer learning comes in handy

`Steps to Use Transfer Learning:`

1. Training a Model to Reuse it
2. Using a Pre Trained Model
3. Extraction of Features
4. Extraction of Features in Neural Networks

In [10]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, confusion_matrix

import tensorflow as tf
from tensorflow import keras
from keras import models, layers
from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import preprocess_input
from keras.applications.efficientnet import EfficientNetB7
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

# Data Preprocessing

`Data Augmentation`

In [11]:
datagen = ImageDataGenerator(preprocess_input,
                             horizontal_flip=True,
                             width_shift_range=0.2,
                             validation_split=0.2,
                             )

In [ ]:
dir_path = './coffee.jpeg'

In [22]:
training_generator = datagen.flow_from_directory(
    dir_path,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    subset='training',
)

validation_generator = datagen.flow_from_directory(
    dir_path,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
)

NotADirectoryError: [Errno 20] Not a directory: './coffee.jpeg'

In [8]:
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224,224,3))

In [9]:
base_model.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [15]:
for layer in base_model.layers:
    layer.trainable = False

# Classifier
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Flatten()(x)
x = layers.Dense(units=512, activation='relu')(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(units=512, activation='relu')(x)
x = layers.Dropout(0.3)(x)
output  = layers.Dense(units=5, activation='softmax')(x)
model = models.Model(base_model.input, output)

In [16]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [17]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [20]:
# callbacks
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min')
checkpoint = keras.callbacks.ModelCheckpoint('./model.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
model_checkpoint = keras.callbacks.ModelCheckpoint('best_weights.h5', monitor='val_acc', save_best_only=True)

In [21]:
history = model.fit(training_generator, validation_data=validation_generator, epochs=2, callbacks=[early_stopping, checkpoint, model_checkpoint])

NameError: name 'training_generator' is not defined